[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/generation/langchain/handbook/03-langchain-conversational-memory.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/generation/langchain/handbook/03-langchain-conversational-memory.ipynb)

#### [LangChain Handbook](https://pinecone.io/learn/langchain)

# Conversational Memory

Conversational memory is how chatbots can respond to our queries in a chat-like manner. It enables a coherent conversation, and without it, every query would be treated as an entirely independent input without considering past interactions.

The memory allows a _"agent"_ to remember previous interactions with the user. By default, agents are *stateless* — meaning each incoming query is processed independently of other interactions. The only thing that exists for a stateless agent is the current input, nothing else.

There are many applications where remembering previous interactions is very important, such as chatbots. Conversational memory allows us to do that.

In this notebook we'll explore this form of memory in the context of the LangChain library.

We'll start by importing all of the libraries that we'll be using in this example.

In [2]:
import inspect
import os
import json
import tiktoken


from langchain.llms import AzureOpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (
    ConversationBufferMemory, 
    ConversationSummaryMemory, 
    ConversationBufferWindowMemory,
    ConversationKGMemory
)
from langchain.callbacks import get_openai_callback

To run this notebook, we will need to use an OpenAI LLM. Here we will setup the LLM we will use for the whole notebook, just input your openai api key when prompted. 

In [3]:
# read from a config.json file
with open("config.json", "r") as f:
    config = json.load(f)

In [4]:
# set the environment variable
os.environ['OPENAI_API_KEY'] = config['AZURE_OPENAI_KEY']
os.environ['OPENAI_API_TYPE'] = config['AZURE_OPENAI_APITYPE']
os.environ['OPENAI_API_VERSION'] = config['AZURE_OPENAI_APIVERSION']
os.environ['OPENAI_API_BASE'] = config['AZURE_OPENAI_ENDPOINT']

In [5]:
llm = AzureOpenAI(
    deployment_name='text-davinci-003',
    model_name='text-davinci-003'
)

In [14]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

Now let's dive into **Conversational Memory**.

## What is memory?

**Definition**: Memory is an agent's capacity of remembering previous interactions with the user (think chatbots)

The official definition of memory is the following:


> By default, Chains and Agents are stateless, meaning that they treat each incoming query independently. In some applications (chatbots being a GREAT example) it is highly important to remember previous interactions, both at a short term but also at a long term level. The concept of “Memory” exists to do exactly that.


As we will see, although this sounds really straightforward there are several different ways to implement this memory capability.

Before we delve into the different memory modules that the library offers, we will introduce the chain we will be using for these examples: the `ConversationChain`.

As always, when understanding a chain it is interesting to peek into its prompt first and then take a look at its `._call` method. As we saw in the chapter on chains, we can check out the prompt by accessing the `template` within the `prompt` attribute.

In [7]:
conversation = ConversationChain(
    llm=llm
)

In [8]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


Interesting! So this chain's prompt is telling it to chat with the user and try to give truthful answers. If we look closely, there is a new component in the prompt that we didn't see when we were tinkering with the `LLMMathChain`: _history_. This is where our memory will come into play.

What is this chain doing with this prompt? Let's take a look.

In [9]:
print(inspect.getsource(conversation._call), inspect.getsource(conversation.apply))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        response = self.generate([inputs], run_manager=run_manager)
        return self.create_outputs(response)[0]
     def apply(
        self, input_list: List[Dict[str, Any]], callbacks: Callbacks = None
    ) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        callback_manager = CallbackManager.configure(
            callbacks, self.callbacks, self.verbose
        )
        run_manager = callback_manager.on_chain_start(
            dumpd(self),
            {"input_list": input_list},
        )
        try:
            response = self.generate(input_list, run_manager=run_manager)
        except (KeyboardInterrupt, Exception) as e:
            run_manager.on_chain_error(e)
            raise e
        outputs = self.create_outputs(response)
        run_manager.on_chain_end({"outputs": output

Nothing really magical going on here, just a straightforward pass through an LLM. In fact, this chain inherits these methods directly from the `LLMChain` without any modification:

In [10]:
print(inspect.getsource(LLMChain._call), inspect.getsource(LLMChain.apply))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        response = self.generate([inputs], run_manager=run_manager)
        return self.create_outputs(response)[0]
     def apply(
        self, input_list: List[Dict[str, Any]], callbacks: Callbacks = None
    ) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        callback_manager = CallbackManager.configure(
            callbacks, self.callbacks, self.verbose
        )
        run_manager = callback_manager.on_chain_start(
            dumpd(self),
            {"input_list": input_list},
        )
        try:
            response = self.generate(input_list, run_manager=run_manager)
        except (KeyboardInterrupt, Exception) as e:
            run_manager.on_chain_error(e)
            raise e
        outputs = self.create_outputs(response)
        run_manager.on_chain_end({"outputs": output

So basically this chain combines an input from the user with the conversation history to generate a meaningful (and hopefully truthful) response.

Now that we've understood the basics of the chain we'll be using, we can get into memory. Let's dive in!

## Memory types

In this section we will review several memory types and analyze the pros and cons of each one, so you can choose the best one for your use case.

### Memory type #1: ConversationBufferMemory

The `ConversationBufferMemory` does just what its name suggests: it keeps a buffer of the previous conversation excerpts as part of the context in the prompt.

**Key feature:** _the conversation buffer memory keeps the previous pieces of conversation completely unmodified, in their raw form._

In [11]:
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

We pass a user prompt the the `ConversationBufferMemory` like so:

In [12]:
conversation_buf("Good morning AI!")

{'input': 'Good morning AI!',
 'history': '',
 'response': " Good morning! It's a beautiful day today. The sun is shining and the sky is a lovely shade of blue. How can I help you?"}

This one call used a total of `85` tokens, but we can't see that from the above. If we'd like to count the number of tokens being used we just pass our conversation chain object and the message we'd like to input via the `count_tokens` function we defined earlier:

In [15]:
count_tokens(
    conversation_buf, 
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 146 tokens


' Interesting! That sounds like a fascinating area of research. Do you have any specific questions about Large Language Models and how they integrate with external knowledge?'

In [16]:
count_tokens(
    conversation_buf,
    "I just want to analyze the different possibilities. What can you think of?"
)

Spent a total of 296 tokens


" Hmm, that's a great question. One possibility is to use a Large Language Model as a tool to create a more comprehensive understanding of a domain by incorporating external knowledge. For example, if you are studying a particular field of medicine, a Large Language Model could be used to augment existing knowledge with additional information from medical journals and other sources. This could help to create a more accurate and comprehensive understanding of the field. Another possibility is to use Large Language Models to generate new insights from existing knowledge. By combining external knowledge with the language model's predictive capabilities, you can generate new insights and form new hypotheses that can be further explored and tested."

In [17]:
count_tokens(
    conversation_buf, 
    "Which data source types could be used to give context to the model?"
)

Spent a total of 416 tokens


" That's a great question. Data sources that can be used to give context to a Large Language Model include text-based sources such as books, journals, and other documents; audio sources such as podcasts and recordings; and image sources such as photographs and videos. Additionally, a Large Language Model can be used to integrate multiple data sources to create a more comprehensive understanding of a domain. For example, you can combine text data with image data to create a better understanding of a particular field of study."

In [18]:
count_tokens(
    conversation_buf, 
    "What is my aim again?"
)

Spent a total of 466 tokens


' Your aim is to explore the potential of integrating Large Language Models with external knowledge. You are looking to analyze different possibilities and data sources that could be used to give context to the model.'

Our LLM with `ConversationBufferMemory` can clearly remember earlier interactions in the conversation. Let's take a closer look to how the LLM is saving our previous conversation. We can do this by accessing the `.buffer` attribute for the `.memory` in our chain.

In [19]:
print(conversation_buf.memory.buffer)

Human: Good morning AI!
AI:  Good morning! It's a beautiful day today. The sun is shining and the sky is a lovely shade of blue. How can I help you?
Human: My interest here is to explore the potential of integrating Large Language Models with external knowledge
AI:  Interesting! That sounds like a fascinating area of research. Do you have any specific questions about Large Language Models and how they integrate with external knowledge?
Human: I just want to analyze the different possibilities. What can you think of?
AI:  Hmm, that's a great question. One possibility is to use a Large Language Model as a tool to create a more comprehensive understanding of a domain by incorporating external knowledge. For example, if you are studying a particular field of medicine, a Large Language Model could be used to augment existing knowledge with additional information from medical journals and other sources. This could help to create a more accurate and comprehensive understanding of the field. A

Nice! So every piece of our conversation has been explicitly recorded and sent to the LLM in the prompt.

### Memory type #2: ConversationSummaryMemory

The problem with the `ConversationBufferMemory` is that as the conversation progresses, the token count of our context history adds up. This is problematic because we might max out our LLM with a prompt that is too large to be processed.

Enter `ConversationSummaryMemory`.

Again, we can infer from the name what is going on.. we will keep a summary of our previous conversation snippets as our history. How will we summarize these? LLM to the rescue.

**Key feature:** _the conversation summary memory keeps the previous pieces of conversation in a summarized form, where the summarization is performed by an LLM._

In this case we need to send the llm to our memory constructor to power its summarization ability.

In [20]:
conversation_sum = ConversationChain(
    llm=llm, 
    memory=ConversationSummaryMemory(llm=llm)
)

When we have an llm, we always have a prompt ;) Let's see what's going on inside our conversation summary memory:

In [21]:
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


Cool! So each new interaction is summarized and appended to a running summary as the memory of our chain. Let's see how this works in practice!

In [22]:
# without count_tokens we'd call `conversation_sum("Good morning AI!")`
# but let's keep track of our tokens:
count_tokens(
    conversation_sum, 
    "Good morning AI!"
)

Spent a total of 279 tokens


" Good morning! It's a pleasure to talk to you. How can I help you today?"

In [23]:
count_tokens(
    conversation_sum, 
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 420 tokens


" That sounds like a fascinating project! I'm happy to help in any way I can. Could you tell me a bit more about the project? What kind of external knowledge do you plan to integrate?"

In [24]:
count_tokens(
    conversation_sum, 
    "I just want to analyze the different possibilities. What can you think of?"
)

Spent a total of 647 tokens


" That's a great idea! I'm sure there are plenty of possibilities for integrating external knowledge with Large Language Models. One possibility could be to use the external knowledge to inform the training of the large language models. For example, you could use the external knowledge to highlight important areas of the training data that the model should pay more attention to. Additionally, you could use the external knowledge to create more sophisticated features and embeddings for the large language models."

In [25]:
count_tokens(
    conversation_sum, 
    "Which data source types could be used to give context to the model?"
)

Spent a total of 778 tokens


' There are many types of data sources that can be used to give context to the model. For example, you could use unstructured data sources such as text corpora or audio recordings to provide additional context, or you could use structured data sources such as knowledge graphs or databases to provide more detailed information about the context. Additionally, you could also use external APIs or web services to access additional data.'

In [26]:
count_tokens(
    conversation_sum, 
    "What is my aim again?"
)

Spent a total of 769 tokens


' Your aim is to explore the potential of integrating Large Language Models with external knowledge.'

In [27]:
print(conversation_sum.memory.buffer)



The human greeted the AI and the AI responded with a polite greeting and offered help. The human then explained their project of exploring the potential of integrating Large Language Models with external knowledge, to which the AI expressed interest and asked for more information about the external knowledge to be integrated. The AI then suggested that there are plenty of possibilities for integrating external knowledge with Large Language Models, such as using the external knowledge to inform the training of the large language models or to create more sophisticated features and embeddings for the large language models. When asked about which data source types could be used to give context to the model, the AI suggested using unstructured data sources such as text corpora or audio recordings, structured data sources such as knowledge graphs or databases, or external APIs or web services to access additional data. The AI reminded the human that their aim was to explore the potential o

You might be wondering.. if the aggregate token count is greater in each call here than in the buffer example, why should we use this type of memory? Well, if we check out buffer we will realize that although we are using more tokens in each instance of our conversation, our final history is shorter. This will enable us to have many more interactions before we reach our prompt's max length, making our chatbot more robust to longer conversations.

We can count the number of tokens being used (without making a call to OpenAI) using the `tiktoken` tokenizer like so:

In [28]:
# initialize tokenizer
tokenizer = tiktoken.encoding_for_model('text-davinci-003')

# show number of tokens for the memory used by each memory type
print(
    f'Buffer memory conversation length: {len(tokenizer.encode(conversation_buf.memory.buffer))}\n'
    f'Summary memory conversation length: {len(tokenizer.encode(conversation_sum.memory.buffer))}'
)

Buffer memory conversation length: 412
Summary memory conversation length: 185


_Practical Note: the `text-davinci-003` and `gpt-3.5-turbo` models [have](https://platform.openai.com/docs/api-reference/completions/create#completions/create-max_tokens) a large max tokens count of 4096 tokens between prompt and answer._

### Memory type #3: ConversationBufferWindowMemory

Another great option for these cases is the `ConversationBufferWindowMemory` where we will be keeping a few of the last interactions in our memory but we will intentionally drop the oldest ones - short-term memory if you'd like. Here the aggregate token count **and** the per-call token count will drop noticeably. We will control this window with the `k` parameter.

**Key feature:** _the conversation buffer window memory keeps the latest pieces of the conversation in raw form_

In [37]:
conversation_bufw = ConversationChain(
    llm=llm, 
    memory=ConversationBufferWindowMemory(k=2)
)

In [38]:
count_tokens(
    conversation_bufw, 
    "Good morning AI!"
)

Spent a total of 97 tokens


" Good morning! It's a beautiful day today. I can see the sun is shining and the sky is a deep blue color. How can I help you?"

In [39]:
count_tokens(
    conversation_bufw, 
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 156 tokens


" That sounds fascinating! I'm not familiar with Large Language Models, but I'm sure I can find some resources to help you. Can you tell me a bit more about your project?"

In [40]:
count_tokens(
    conversation_bufw, 
    "I just want to analyze the different possibilities. What can you think of?"
)

Spent a total of 247 tokens


" Well, one possibility is to use the Large Language Models to build a knowledge graph, which can then be used to provide more accurate results from natural language queries. Another option is to use the Large Language Models to supplement external knowledge by providing additional context for better understanding. Do either of those options sound like something you'd be interested in exploring further?"

In [41]:
count_tokens(
    conversation_bufw, 
    "Which data source types could be used to give context to the model?"
)

Spent a total of 297 tokens


' You can use a variety of data source types to give context to the model, including structured data such as databases, unstructured data such as text documents, and even multimedia information like audio or video. Additionally, you can use machine learning algorithms to extract relevant information from the data sources, which can then be used to provide context to the Large Language Model.'

In [42]:
count_tokens(
    conversation_bufw, 
    "What is my aim again?"
)

Spent a total of 277 tokens


' Your aim is to analyze different possibilities for using Large Language Models to build a knowledge graph and provide more accurate results from natural language queries.'

As we can see, it effectively 'fogot' what we talked about in the first interaction. Let's see what it 'remembers'. Given that we set k to be `1`, we would expect it remembers only the last interaction.

We need to access a special method here since, in this memory type, the buffer is first passed through this method to be sent later to the llm.

In [43]:
bufw_history = conversation_bufw.memory.load_memory_variables(
    inputs=[]
)['history']

In [44]:
print(bufw_history)

Human: Which data source types could be used to give context to the model?
AI:  You can use a variety of data source types to give context to the model, including structured data such as databases, unstructured data such as text documents, and even multimedia information like audio or video. Additionally, you can use machine learning algorithms to extract relevant information from the data sources, which can then be used to provide context to the Large Language Model.
Human: What is my aim again?
AI:  Your aim is to analyze different possibilities for using Large Language Models to build a knowledge graph and provide more accurate results from natural language queries.


Makes sense. 

On the plus side, we are shortening our conversation length when compared to buffer memory _without_ a window:

In [45]:
print(
    f'Buffer memory conversation length: {len(tokenizer.encode(conversation_buf.memory.buffer))}\n'
    f'Summary memory conversation length: {len(tokenizer.encode(conversation_sum.memory.buffer))}\n'
    f'Buffer window memory conversation length: {len(tokenizer.encode(bufw_history))}'
)

Buffer memory conversation length: 412
Summary memory conversation length: 185
Buffer window memory conversation length: 132


_Practical Note: We are using `k=2` here for illustrative purposes, in most real world applications you would need a higher value for k._

### More memory types!

Given that we understand memory already, we will present a few more memory types here and hopefully a brief description will be enough to understand their underlying functionality.

#### ConversationSummaryBufferMemory

**Key feature:** _the conversation summary memory keeps a summary of the earliest pieces of conversation while retaining a raw recollection of the latest interactions._

#### ConversationKnowledgeGraphMemory

This is a super cool memory type that was introduced just [recently](https://twitter.com/LangChainAI/status/1625158388824043522). It is based on the concept of a _knowledge graph_ which recognizes different entities and connects them in pairs with a predicate resulting in (subject, predicate, object) triplets. This enables us to compress a lot of information into highly significant snippets that can be fed into the model as context. If you want to understand this memory type in more depth you can check out [this](https://apex974.com/articles/explore-langchain-support-for-knowledge-graph) blogpost.

**Key feature:** _the conversation knowledge graph memory keeps a knowledge graph of all the entities that have been mentioned in the interactions together with their semantic relationships._

In [69]:
# you may need to install this library
# !pip install -qU networkx

In [ ]:
conversation_kg = ConversationChain(
    llm=llm, 
    memory=ConversationKGMemory(llm=llm)
)

In [71]:
count_tokens(
    conversation_kg, 
    "My name is human and I like mangoes!"
)

Spent a total of 1565 tokens


" Hi Human! My name is AI. It's nice to meet you. I like mangoes too! Did you know that mangoes are a great source of vitamins A and C?"

The memory keeps a knowledge graph of everything it learned so far.

In [72]:
conversation_kg.memory.kg.get_triples()

[('human', 'human', 'name'), ('human', 'mangoes', 'likes')]

#### ConversationEntityMemory

**Key feature:** _the conversation entity memory keeps a recollection of the main entities that have been mentioned, together with their specific attributes._

The way this works is quite similar to the `ConversationKnowledgeGraphMemory`, you can refer to the [docs](https://python.langchain.com/en/latest/modules/memory/types/entity_summary_memory.html) if you want to see it in action. 

## What else can we do with memory?

There are several cool things we can do with memory in langchain. We can:
* implement our own custom memory module
* use multiple memory modules in the same chain
* combine agents with memory and other tools

If this piques your interest, we suggest you to go take a look at the memory [how-to](https://langchain.readthedocs.io/en/latest/modules/memory/how_to_guides.html) section in the docs!